# Import modules

In [ ]:
# Import extensions
import sys 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path
from IPython.display import FileLink, FileLinks
# FileLinks('.') #lists all downloadable files on server
import cattr

In [ ]:
sys.path.append('..')

In [ ]:
import attr
import cattr
import json
from pathlib import Path
import typing

In [ ]:
import barely_db
barely_db._reload_module()
from barely_db import *
# test_BUID()

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
base_path_raw = 'C:\\Repo\\barely-db\\tests\\Database'
# base_path_raw = 'C:\\Repo\\__code_dev_db\\ma-standalone\\env_mtools\\src\\manufacturing-tools\\tests\\Database'
# base_path_raw = 'G:\\Shared drives\\Database'
base_path = str(Path(base_path_raw))

In [ ]:
bdb = BarelyDB(base_path=base_path_raw)
bdb.load_entities()

In [ ]:
bdb.config.save(bdb.base_path)

In [ ]:
bdb.path_depth

In [ ]:
bdb.known_bases

In [ ]:
config = BarelyDBConfig()
config

In [ ]:
sys_config = BarelyDBSystemConfig.from_legacy_default()
sys_config

In [ ]:
sys_config.save()

In [ ]:
sys_config = None

In [ ]:
sys_config = BarelyDBSystemConfig.load()
sys_config

In [ ]:
bdb = BarelyDB(name='battrion_manufacturing')
bdb